In [5]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import datetime
#graphing
import matplotlib.pyplot as plt
#stats
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

#import testing
import sys
sys.path.append("../")
import selection_tests

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
#format the bus .dat from augirregabiria and mira's website
data = np.fromfile('bus1234.dat')
data = data.reshape(int(len(data)/6),6)
data = pd.DataFrame(data,columns=['id','group','year','month','replace','miles'])

#save to .csv
data.to_csv('bus1234.csv')

#divide by 1e6 (use the same scale are Rust and AM)
data['miles'] = (data['miles'])/1e6

#switch to date time for ease 
data['date'] = pd.to_datetime(data[['year', 'month']].assign(Day=1))
data = data[['id','group','date','replace','miles']]

#lag date
date_lag = data.copy()
date_lag['date'] = date_lag['date'] - pd.DateOffset(months=1)
data = data.merge(date_lag, how='left', on=['id','group','date'] , suffixes=('','_next'))
data = data.dropna()

In [8]:
#define cost functon using lambda expression
linear_cost = lambda params, x, i: (1-i)*x*params[i] + i*params[i]

linear_model = selection_tests.CCP(data['replace'], data['miles'], data['miles_next'], ['theta1','RC'], linear_cost)
print(linear_model.results.summary())

NameError: name 'states' is not defined